In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from kerastuner.tuners import RandomSearch

import warnings
warnings.filterwarnings("ignore")
#!pip install iterative-stratification

import tensorflow as tf
from keras.utils import plot_model
import sklearn

In [ ]:
os.listdir(os.getcwd())

In [ ]:
os.chdir("../input/iterstrat")
os.getcwd()

In [ ]:
from ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
os.chdir("../input/iterstrat")
from ml_stratifiers import MultilabelStratifiedKFold,RepeatedMultilabelStratifiedKFold,MultilabelStratifiedShuffleSplit

In [ ]:
train_features = pd.read_csv("/kaggle/input/lish-moa/train_features.csv")
train_targets_scored = pd.read_csv("/kaggle/input/lish-moa/train_targets_scored.csv")
#train_targets_unscored = pd.read_csv("/kaggle/input/lish-moa/train_targets_nonscored.csv")

In [ ]:
COLS = ['cp_type','cp_dose']
FE = []
for col in COLS:
    for mod in train_features[col].unique():
        FE.append(mod)
        train_features[mod] = (train_features[col] == mod).astype(int)
del train_features['sig_id']
del train_features['cp_type']
del train_features['cp_dose']
FE+=list(train_features.columns) 
del train_targets_scored['sig_id']

In [ ]:
X = np.array(train_features.to_numpy(), dtype=np.float)
y = np.array(train_targets_scored.to_numpy(), dtype=np.float)

In [ ]:
def add_layers(x,unit,drop):
    x = tf.keras.layers.Dense(units=unit)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Dropout(drop)(x)
    return x

In [ ]:
def build_model():
    inp = tf.keras.layers.Input(shape=(train_features.shape[1],))
    x = tf.keras.layers.BatchNormalization()(inp)
    for i in [2048,1024,512]:
        x = add_layers(x,i,0.4)
    out = tf.keras.layers.Dense(206, activation="sigmoid")(x)
    model = tf.keras.models.Model(inputs=inp,outputs=out)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy","binary_crossentropy"])
    return model
model = build_model()
model.summary()

In [ ]:
# from sklearn.model_selection import GridSearchCV
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# model = KerasClassifier(build_fn=build_model, verbose=0)

In [ ]:
# batch_size = [60, 100]
# epochs = [10, 50, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)
# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
# grid_result = grid.fit(X, y)
# # summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

# cross validation schema

In [ ]:
from skmultilearn.model_selection import iterative_train_test_split
X_train, y_train, X_val, y_val = iterative_train_test_split(X, y, test_size = 0.1)
X_train.shape,y_train.shape,X_val.shape,y_val.shape

In [ ]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import numpy as np


mskf = MultilabelStratifiedKFold(n_splits=2, shuffle=True)

for train_index, test_index in mskf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(X_train,y_test,epochs=50,batch_size=60,verbose=0)
    pred = model.predict(X_test)
    print(round(tf.keras.losses.binary_crossentropy(y_test, pred).numpy().mean(),6))

In [ ]:
from iterstrat.ml_stratifiers import RepeatedMultilabelStratifiedKFold

rmskf = RepeatedMultilabelStratifiedKFold(n_splits=2,n_repeats=2, random_state=0)

for train_index, test_index in rmskf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(X_train,y_test,epochs=50,batch_size=60,verbose=0)
    pred = model.predict(X_test)
    print(round(tf.keras.losses.binary_crossentropy(y_test, pred).numpy().mean(),6))

In [ ]:
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

msss  = MultilabelStratifiedShuffleSplit(n_splits=3, test_size=0.5, random_state=0)

for train_index, test_index in msss.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(X_train,y_test,epochs=50,batch_size=60,verbose=0)
    pred = model.predict(X_test)
    print(round(tf.keras.losses.binary_crossentropy(y_test, pred).numpy().mean(),6))

# testing with leader board

In [ ]:
test_features = pd.read_csv("/kaggle/input/lish-moa/test_features.csv")

sample_submission = pd.read_csv("/kaggle/input/lish-moa/sample_submission.csv")
COLS = ['cp_type','cp_dose']
FE = []
for col in COLS:
    for mod in test_features[col].unique():
        FE.append(mod)
        test_features[mod] = (test_features[col] == mod).astype(int)
del test_features['sig_id']
del test_features['cp_type']
del test_features['cp_dose']

In [ ]:
pred = model.predict(test_features)

In [ ]:
sub = pd.DataFrame(data=pred, columns=train_targets_scored.columns)
sub.insert(0, column = 'sig_id', value=sample_submission['sig_id'])

In [ ]:
sub.to_csv('submission.csv',index=False)